# 한정된 데이터로 Transfer learning 적용해보기

# 1. CT이미지 데이터셋 살펴보기

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

In [ ]:
data_dir = "./DATASET/Segmentation/"
data_df = pd.read_csv(os.path.join(data_dir, "train.csv"))

In [ ]:
data_df

In [ ]:
def extract_client_id(x):
    return x.split('-')[0]

In [ ]:
data_df['id'] = data_df.ImageId.apply(lambda x: extract_client_id(x))
data_df

In [ ]:
def get_client_data(data_df, index):
    client_ids = np.unique(data_df.id.values)
    client_id = client_ids[index]
    client_data = data_df[data_df.id == client_id]
    Image_files = list(client_data['ImageId'])
    mask_files = list(client_data['MaskId'])
    return client_id, Image_files, mask_files

In [ ]:
index = 100
get_client_data(data_df, index)

In [ ]:
regions = ["background", "trachea", "heart", "lung"]
colors = ((0,0,0), (255,0,0), (0,255,0), (0,0,255))

In [ ]:
index = 50
client_id, image_files , mask_files = get_client_data(data_df, index)

canvas = np.zeros(shape=(512, 2*512+50, 3), dtype=np.uint8)
for i in range(len(image_files)):
    image = cv2.imread(os.path.join(data_dir, "images", image_files[i]))
    mask = cv2.imread(os.path.join(data_dir, "masks", mask_files[i]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    mask[mask<240] = 0
    mask[mask>=240] = 255
    
    canvas[:, 0:512, :] = image
    canvas[:, 512+50:2*512+50, :] = mask
    
    cv2.imshow('image', canvas)
    key = cv2.waitKey(100)
    if key == 27:
        break
    if key == ord('s'):
        cv2.waitKey(0)
cv2.destroyAllWindows()

# 2. 데이터셋 구축과 연산을 위한 텐서변환 모듈 작성하기

# 3. VGG16 Backbone을 이용한 U-Net 아키텍쳐 구현해보기

# 4. Semantic segmentation Loss와 학습코드 작성하기

# 5. Weight Initialization 과 Transfer learning 모델 비교하기

## 5-1 He initialization